# Setup

In [ ]:
import os
import logging as log
from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from holoviews.operation.datashader import datashade, rasterize
from holoviews.operation import decimate

from wp.notebook import WorkloadNotebookAnalysis, trim_wa_path, ptable

hv.extension('bokeh')
hv.renderer('bokeh').theme = built_in_themes['dark_minimal']
hv.renderer('bokeh').webgl = True
pio.templates.default = "plotly"
pio.templates.default = "plotly_dark"

color_cycle = hv.Cycle(['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'])

opts.defaults(
    opts.Curve(tools=['hover'], show_grid=True, color=color_cycle, muted_alpha=0),
    opts.Table(bgcolor='black')
)

## Runs

In [ ]:
ft = WorkloadNotebookAnalysis('/home/kajpuc01/power/pixel6/fortnite/', [
    'fortnite_product_baseline_10_2007',
    'fortnite_product_arm_vh_sched_debug_10_2007'
])

ft.show()

# FPS

## FPS line plots

In [ ]:
ft.plot.results_line(['fps', 'total_frames', 'janks_pc', 'missed_vsync'], height=500, width=800, title='FPS metrics per iteration')

## FPS bar plots

In [ ]:
ft.plot.results_bar(['fps', 'total_frames', 'janks_pc', 'missed_vsync'], height=600, columns=4, title='gmean FPS metrics')

## FPS over time

In [ ]:
# TODO

# Overutilized

## Line plot

In [ ]:
ft.plot.overutilized_line()

# Power meter

## Line plot across iterations

In [ ]:
ft.plot.power_meter_line()

## Bar plot - gmean

In [ ]:
ft.plot.power_meter_bar()

# Capacity

In [ ]:
ft.load_combined_analysis('capacity.pqt', allow_missing=True)

display(ft.analysis['capacity'])

## Line plot

In [ ]:
ds = hv.Dataset(ft.analysis['capacity'].query("iteration == 7 and cpu == 6"), ['time_it', hv.Dimension('wa_path', values=ft.wa_paths), 'iteration', 'cpu'], ['capacity'])
layout_cap = ds.to(hv.Curve, 'time_it', 'capacity').overlay('wa_path').opts(width=2100, height=600)
layout_cap

# uclamp

## Per-task uclamp over time

In [ ]:
ft.plot.uclamp_per_task_line()

# Task placement (activations)

## Overview

### Line plots - counts

In [ ]:
ft.plot.task_activations_stats_count_line(tasks=['GameThread', 'RHIThread', 'RenderThread 0', 'surfaceflinger', 'mali_jd_thread', 'RenderEngine'])

### Line plots - durations

In [ ]:
ft.plot.task_activations_stats_duration_line(tasks=['GameThread', 'RHIThread', 'RenderThread 0', 'surfaceflinger', 'mali_jd_thread', 'RenderEngine'])

### Bar plot - counts

In [ ]:
ft.plot.task_activations_stats_count_bar(tasks=['GameThread', 'RHIThread', 'RenderThread 0', 'surfaceflinger', 'mali_jd_thread', 'RenderEngine'])

### Bar plot - durations

In [ ]:
ft.plot.task_activations_stats_duration_bar(tasks=['GameThread', 'RHIThread', 'RenderThread 0', 'surfaceflinger', 'mali_jd_thread', 'RenderEngine'])